# 04: Train Meta-Ensemble

Train meta-gating ensemble that combines specialist models.

In [ ]:
# Setup
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path().absolute().parent.parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from models import XGBoostModel, LightGBMModel, SentimentModel, RuleBasedModel
from ensemble import MetaEnsemble
from utils.config import PROCESSED_DATA_DIR, SPECIALIST_MODELS_DIR, META_MODEL_DIR

TICKER = "AAPL"

In [ ]:
# Load all trained specialists
xgb = XGBoostModel()
xgb.load(SPECIALIST_MODELS_DIR / f"{TICKER}_xgb.model")

lgb = LightGBMModel()
lgb.load(SPECIALIST_MODELS_DIR / f"{TICKER}_lgb.txt")

sentiment = SentimentModel()
sentiment.load(SPECIALIST_MODELS_DIR / f"{TICKER}_sentiment.pkl")

rule = RuleBasedModel()
rule.load(SPECIALIST_MODELS_DIR / f"{TICKER}_rule.pkl")

specialists = [xgb, lgb, sentiment, rule]
print(f"Loaded {len(specialists)} specialist models")

In [ ]:
# Load features
features = pd.read_csv(PROCESSED_DATA_DIR / f"{TICKER}_features.csv", index_col=0, parse_dates=True)

feature_cols = [col for col in features.columns 
                if col not in ['target_return_1d', 'target_direction', 'open', 'high', 'low', 'close', 'volume']]
X = features[feature_cols].fillna(0)
y = features['target_direction']

mask = y != 0
X = X[mask]
y = y[mask]

print(f"Training meta-ensemble on {len(X)} samples")

In [ ]:
# Extract market features for meta-model
vol_cols = [col for col in features.columns if 'volatility' in col.lower()]
news_cols = [col for col in features.columns if 'news' in col.lower()]

market_features = pd.DataFrame(index=X.index)
if vol_cols:
    market_features['volatility'] = X[vol_cols[0]]
if news_cols:
    market_features['news_intensity'] = X[news_cols[0]]

market_features = market_features.fillna(0)

In [ ]:
# Train meta-ensemble
ensemble = MetaEnsemble(specialists, min_confidence=0.6)
meta_metrics = ensemble.train(X, y, market_features)
print(f"Meta-ensemble metrics: {meta_metrics}")

# Save
ensemble.save(META_MODEL_DIR / f"{TICKER}_meta_ensemble.pkl")
print(f"✓ Saved meta-ensemble")